In [1]:
import requests
import json
import pandas as pd
import datetime
import matplotlib
from pandas import read_csv
from matplotlib import pyplot
import math
import yfinance as yf
import numpy as np

def fundamental(symbol='AAPL'):
        url = 'https://eodhistoricaldata.com/api/{metric}/{sticker}?api_token=5fd0ffd5a187a3.28352247'.replace('{sticker}', symbol)
        url = url.replace('{metric}','fundamentals')
        fundamental_data = requests.get(url)
        #print('fundamental get')
        return json.loads(fundamental_data.text)

In [2]:
aapl=fundamental()
aapl.keys()

dict_keys(['General', 'Highlights', 'Valuation', 'SharesStats', 'Technicals', 'SplitsDividends', 'AnalystRatings', 'Holders', 'InsiderTransactions', 'ESGScores', 'outstandingShares', 'Earnings', 'Financials'])

In [3]:
aapl['Technicals']

{'Beta': 1.2222,
 '52WeekHigh': 157.26,
 '52WeekLow': 106.6342,
 '50DayMA': 146.5957,
 '200DayMA': 138.939,
 'SharesShort': 101107412,
 'SharesShortPriorMonth': 100926641,
 'ShortRatio': 1.18,
 'ShortPercent': 0.0061}

In [4]:
aapl['Valuation']

{'TrailingPE': 29.119,
 'ForwardPE': 25.974,
 'PriceSalesTTM': 7.0824,
 'PriceBookMRQ': 37.2469,
 'EnterpriseValue': 2496290881536,
 'EnterpriseValueRevenue': 7.0698,
 'EnterpriseValueEbitda': 21.4419}

In [5]:
aapl['Earnings'].keys()

dict_keys(['History', 'Trend', 'Annual'])

In [6]:
pd.DataFrame.from_dict(aapl['Earnings']['Annual'],orient='index').head(5)

,date,epsActual
2021-09-30,2021-09-30,4.38
2020-09-30,2020-09-30,3.27
2019-09-30,2019-09-30,2.98
2018-09-30,2018-09-30,2.97
2017-09-30,2017-09-30,2.30


In [7]:
pd.DataFrame.from_dict(aapl['Earnings']['History'], orient='index').loc['2018':'2016']

,reportDate,date,beforeAfterMarket,currency,epsActual,epsEstimate,epsDifference,surprisePercent
2017-12-31,2018-02-01,2017-12-31,AfterMarket,USD,0.9725,0.9650,0.0075,0.7772
2017-09-30,2017-11-02,2017-09-30,AfterMarket,USD,0.5175,0.4675,0.0500,10.6952
2017-06-30,2017-08-01,2017-06-30,AfterMarket,USD,0.4175,0.3925,0.0250,6.3694
2017-03-31,2017-05-02,2017-03-31,AfterMarket,USD,0.5250,0.5050,0.0200,3.9604
2016-12-31,2017-01-31,2016-12-31,AfterMarket,USD,0.8400,0.8025,0.0375,4.6729
2016-09-30,2016-10-25,2016-09-30,AfterMarket,USD,0.4175,0.4150,0.0025,0.6024
2016-06-30,2016-07-26,2016-06-30,AfterMarket,USD,0.3550,0.3450,0.0100,2.8986
2016-03-31,2016-04-26,2016-03-31,AfterMarket,USD,0.4750,0.5000,-0.0250,-5.0000


Can compute Trailing eps (price/Trailing eps), pe growth rate, pe surprisePercent.

In [8]:
aapl['Highlights']

{'MarketCapitalization': 2458702053376,
 'MarketCapitalizationMln': 2458702.0534,
 'EBITDA': 110934999040,
 'PERatio': 29.119,
 'PEGRatio': 1.9839,
 'WallStreetTargetPrice': 168.23,
 'BookValue': 3.882,
 'DividendShare': 0.835,
 'DividendYield': 0.0057,
 'EarningsShare': 5.108,
 'EPSEstimateCurrentYear': 5.59,
 'EPSEstimateNextYear': 5.67,
 'EPSEstimateNextQuarter': 1.11,
 'EPSEstimateCurrentQuarter': 1,
 'MostRecentQuarter': '2021-06-30',
 'ProfitMargin': 0.25,
 'OperatingMarginTTM': 0.2879,
 'ReturnOnAssetsTTM': 0.193,
 'ReturnOnEquityTTM': 1.2713,
 'RevenueTTM': 347155005440,
 'RevenuePerShareTTM': 20.61,
 'QuarterlyRevenueGrowthYOY': 0.364,
 'GrossProfitTTM': 104956000000,
 'DilutedEpsTTM': 5.108,
 'QuarterlyEarningsGrowthYOY': 1}

Can use MarketCap to compute log(MarketCap), liquidity

In [9]:
aapl['Financials'].keys()

dict_keys(['Balance_Sheet', 'Cash_Flow', 'Income_Statement'])

In [ ]:
aapl['Financials']['IncomeStatement']

Can use income statement to compute ps/revenue growth rate

#### Sector

In [62]:
aapl['General']['Sector'],aapl['General']['Industry']

('Technology', 'Consumer Electronics')

In [17]:
ticker='AAPL'
fdmt=fundamental(ticker)
# pd.DataFrame.from_dict(fdmt['Earnings']['Annual'],orient='index').head(5)
earnings=pd.DataFrame.from_dict(fdmt['Earnings']['History'], orient='index').loc['2018':'2016']
eps=earnings['epsActual'][:4].sum()
eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()

In [18]:
eps, eps_growth

(2.4325, 0.1652694610778444)

In [23]:
revenues=pd.DataFrame.from_dict(aapl['Financials']['Income_Statement']['yearly'], orient='index').loc['2018':'2016']['totalRevenue']
revenue_growth=(float(revenues[0])-float(revenues[1]))/float(revenues[1])

In [24]:
revenue_growth

0.06304518199398068

Market Cap

In [52]:
# df4 = yf.download(['AAPL'], start = "2018-01-01", end = "2021-10-15")
# pre_price=df4['Open'][0]
# cur_price=df4['Open'][-1]
# cur_marketCap=float(fdmt['Highlights']['MarketCapitalization'])
# pre_marketCap=cur_marketCap*pre_price/cur_price
# pre_marketCap, cur_marketCap

[*********************100%***********************]  1 of 1 completed


(736001598426.6011, 2458702053376.0)

#### Growth/Value?
growth if revenue growth rate >= 30%
value if pe <= 18
none if none
both if both

#### std, return/std

#### Now we record all the data

In [8]:
dfFeatures=pd.DataFrame(columns=['Symbol', 'PE', 'EPSGrowth',
                                 'RevenueGrowth', 'logMarketCap', 'PS','Sector','Sharpe','std','LiquidityRate', 'CorWithBenchMark'])

In [9]:
features=['Symbol', 'PE', 'EPSGrowth',
                                 'RevenueGrowth', 'logMarketCap', 'PS','Sector','Sharpe','std','LiquidityRate', 'CorWithBenchMark']

In [10]:
# allData=pd.concat([pd.read_csv('momentumeEffect.csv'), pd.read_csv("momentumeEffect5.csv")])
allData=pd.read_csv("BigCapMomentum2019.csv")

In [11]:
listOfStocks=allData.drop_duplicates(['ticker'], keep='first')['ticker']

In [12]:
listOfStocks

0       ABMD
1       AAPL
2          A
3        ABT
4       ABBV
        ... 
1089     ZNH
1090    ZNGA
1091      ZS
1092     ZTO
1093     ZTS
Name: ticker, Length: 1094, dtype: object

In [13]:

benchMark = yf.download(['DJIA'], start = "2018-01-01", end = "2019-01-01")
benchMark
from scipy.stats import pearsonr
benchMark_log=np.log((benchMark['Close']/benchMark['Close'].shift(1)).dropna())

for ticker in listOfStocks:
    try:
        fdmt=fundamental(ticker)
        # pd.DataFrame.from_dict(fdmt['Earnings']['Annual'],orient='index').head(5)
        earnings=pd.DataFrame.from_dict(fdmt['Earnings']['History'], orient='index').loc['2018':'2016']
        eps=earnings['epsActual'][:4].sum()
        eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()

        #revenue_growth, eps, eps_growth
        revenues=pd.DataFrame.from_dict(fdmt['Financials']['Income_Statement']['yearly'], orient='index').loc['2018':'2016']['totalRevenue']
        revenue_growth=(float(revenues[0])-float(revenues[1]))/float(revenues[1])

        #market_cap
        df4 = yf.download([ticker], start = "2019-01-01", end = "2021-10-15")
        pre_price=df4['Open'][0]
        cur_price=df4['Open'][-1]
        cur_marketCap=float(fdmt['Highlights']['MarketCapitalization'])
        pre_marketCap=cur_marketCap*pre_price/cur_price
        pre_marketCap, cur_marketCap

        #ps
        ps=pre_marketCap/float(revenues[0])
        print('ps=',ps)

        #pe
        pe=float(df4['Open'][0])/eps
        print('pe=',pe)

        #sector
        sector=fdmt['General']['Sector']

        #sharpe, std
        df3 = yf.download([ticker], start = "2018-01-01", end = "2019-01-01")
        # df3 = df3.div(df3['Open'][0])
        std= np.log((df3['Close']/df3['Close'].shift(1)).dropna()).std()*np.sqrt(252)
        buyHoldReturn=(df3['Close'][-1]-df3['Close'][0])/df3['Close'][0]
        Sharpe=buyHoldReturn/std
        Sharpe, std

        #liquidity rate
        liquidityRate=100*(df3['Volume']*df3['Open']).mean()/pre_marketCap


        #correlation
        benchMark_log
        stock_log=np.log((df3['Close']/df3['Close'].shift(1)).dropna())
        common_Length=min(len(stock_log), len(benchMark_log))
        cor,_ = pearsonr(stock_log[:common_Length], benchMark_log[:common_Length])

        new_row=pd.DataFrame([[ticker, pe, eps_growth, revenue_growth, int(np.log10(pre_marketCap)), ps, sector, Sharpe, std, liquidityRate, cor]], 
                             columns=features)
        # print(new_row)
        dfFeatures=pd.concat([dfFeatures, new_row])
    except:
        None
# dfFeatures
# dfFeatures=dfFeatures.drop_duplicates(['Symbol'], keep='first')
# dfFeatures.to_csv('BigCapFeatures2019.csv')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DJIA: None
[*********************100%***********************]  1 of 1 completed
ps= 32.89874119547899
pe= 158.6203446337013
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 2.930040960228006
pe= 15.918807748165307
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 4.678831798671366
pe= 28.059071729957804
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 4.796959424423734
pe= 28.155999755859376
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 5.738218850066342
pe= 16.322003195665392
[*********************100%***********************]  1

C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:12: RuntimeWarning: invalid value encountered in double_scalars
  eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()


[*********************100%***********************]  1 of 1 completed
ps= 1.3146264791729305
pe= 9.253111716622634
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:12: RuntimeWarning: invalid value encountered in double_scalars
  eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()
C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:31: RuntimeWarning: divide by zero encountered in double_scalars
  pe=float(df4['Open'][0])/eps



ps= 1.6102574102187583
pe= inf
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 92.79834751771043
pe= -13.179298161138227
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 0.7491846942407075
pe= 10.4256840138319
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 2.0035767204039265
pe= 9.85846182016226
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 4.380359346558754
pe= 112.56250143051149
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 3.7221086618942265
pe= 25.570880407574535
[*********************100%***********************]  1

...


[*********************100%***********************]  1 of 1 completed
ps= 0.09994279238667793
pe= 33.15640119781701
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 4.641303208467769
pe= 30.67003243700021
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 2.0085821702784807
pe= 23.401408128335444
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 0.4046518421330618
pe= 9.453397571614689
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 6.674818900080111
pe= 58.2830177163178
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 2.8626811050

C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:12: RuntimeWarning: invalid value encountered in double_scalars
  eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()


[*********************100%***********************]  1 of 1 completed
ps= 2.482883541976535
pe= 13.996503069684223
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 0.11770438633814727
pe= 293.76825780227796
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 5.219529732808252
pe= 34.94623741368667
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 43.486089700071766
pe= inf
[*********************100%***********************]  1 of 1 completed

C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:12: RuntimeWarning: invalid value encountered in double_scalars
  eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()
C:\Users\tao\AppData\Local\Temp/ipykernel_20036/236656064.py:31: RuntimeWarning: divide by zero encountered in double_scalars
  pe=float(df4['Open'][0])/eps



[*********************100%***********************]  1 of 1 completed
ps= 0.9909595018510117
pe= 24.927908394131208
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ps= 7.831523184218679
pe= 35.21250089009603
[*********************100%***********************]  1 of 1 completed


In [7]:
dfFeatures

NameError: name 'dfFeatures' is not defined

In [6]:
dfFeatures=dfFeatures.drop_duplicates(['Symbol'], keep='first')
dfFeatures
dfFeatures.to_csv('BigCapFeatures2019.csv')

NameError: name 'dfFeatures' is not defined

In [31]:
dfFeatures.to_csv('BigCapFeatures.csv')

In [93]:
ticker='CVS'
fdmt=fundamental(ticker)
# pd.DataFrame.from_dict(fdmt['Earnings']['Annual'],orient='index').head(5)
earnings=pd.DataFrame.from_dict(fdmt['Earnings']['History'], orient='index').loc['2018':'2016']
eps=earnings['epsActual'][:4].sum()
eps_growth=(earnings['epsActual'][:4].sum()-earnings['epsActual'][-4:].sum())/earnings['epsActual'][-4:].sum()

#revenue_growth, eps, eps_growth
revenues=pd.DataFrame.from_dict(fdmt['Financials']['Income_Statement']['yearly'], orient='index').loc['2018':'2016']['totalRevenue']
revenue_growth=(float(revenues[0])-float(revenues[1]))/float(revenues[1])
print('rev',revenues[0])

#market_cap
df4 = yf.download([ticker], start = "2018-01-01", end = "2021-10-15")
pre_price=df4['Open'][0]
cur_price=df4['Open'][-1]
cur_marketCap=float(fdmt['Highlights']['MarketCapitalization'])
pre_marketCap=cur_marketCap*pre_price/cur_price
pre_marketCap, cur_marketCap
print('pre_mar=',pre_marketCap)

#ps
ps=pre_marketCap/float(revenues[0])
print('ps=',ps)

#pe
pe=float(df4['Open'][0])/eps
print('pe=',pe)

#sector
sector=fdmt['General']['Sector']

#sharpe, std
df3 = yf.download([ticker], start = "2017-01-01", end = "2018-01-01")
# df3 = df3.div(df3['Open'][0])
std= np.log((df3['Close']/df3['Close'].shift(1)).dropna()).std()*np.sqrt(252)
buyHoldReturn=(df3['Close'][-1]-df3['Close'][0])/df3['Close'][0]
Sharpe=buyHoldReturn/std
Sharpe, std

#liquidity rate
liquidityRate=100*(df3['Volume']*df3['Open']).mean()/pre_marketCap


#correlation
benchMark_log
stock_log=np.log((df3['Close']/df3['Close'].shift(1)).dropna())
common_Length=min(len(stock_log), len(benchMark_log))
cor,_ = pearsonr(stock_log[:common_Length], benchMark_log[:common_Length])

new_row=pd.DataFrame([[ticker, pe, eps_growth, revenue_growth, int(np.log10(pre_marketCap)), ps, sector, Sharpe, std, liquidityRate, cor]], 
                     columns=features)
# print(new_row)
dfFeatures=pd.concat([dfFeatures, new_row])

rev 184765000000.00
[*********************100%***********************]  1 of 1 completed
pre_mar= 97036485032.91109
ps= 0.5251886722751121
pe= 12.341215803816512
[*********************100%***********************]  1 of 1 completed


#### Correlation With Benchmark

In [40]:
benchMark = yf.download(['DJIA'], start = "2017-01-01", end = "2018-01-01")
benchMark
from scipy.stats import pearsonr
benchMark_log=np.log((benchMark['Close']/benchMark['Close'].shift(1)).dropna())
benchMark_log
stock_log=np.log((df3['Close']/df3['Close'].shift(1)).dropna())
cor,_ = pearsonr(stock_log, benchMark_log)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,19872.859375,20021.599609,19694.359375,19881.759766,19881.759766,3770530000
2017-01-04,19890.939453,20037.419922,19812.150391,19942.160156,19942.160156,3764890000
2017-01-05,19922.560547,20031.400391,19747.599609,19899.289062,19899.289062,3761820000
2017-01-06,19906.960938,20048.519531,19781.189453,19963.800781,19963.800781,3339890000
2017-01-09,19931.410156,20021.529297,19807.519531,19887.380859,19887.380859,3217610000
...,...,...,...,...,...,...
2017-12-22,24764.039062,24878.849609,24618.410156,24754.060547,24754.060547,2399830000
2017-12-26,24715.839844,24860.609375,24641.140625,24746.210938,24746.210938,1968780000
2017-12-27,24766.519531,24860.949219,24671.740234,24774.300781,24774.300781,2202080000


In [49]:
# from scipy.stats import pearsonr
# benchMark_log=np.log((benchMark['Close']/benchMark['Close'].shift(1)).dropna())
# benchMark_log
# stock_log=np.log((df3['Close']/df3['Close'].shift(1)).dropna())
# stock_log.min()
# cor,_ = pearsonr(stock_log, benchMark_log)
# cor

0.4054855460585214